In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# ==========================================
# 1. LOAD DATA & PREPROCESSING
# ==========================================
print("Sedang memproses data...")
df = pd.read_csv('Latest 2025 movies Datasets.csv')

# Membersihkan nilai kosong (NaN)
df = df.dropna(subset=['release_date', 'vote_average', 'popularity', 'vote_count'])
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
df = df.dropna(subset=['release_date'])

# Feature Engineering: Mengambil Tahun & Bulan
df['release_year'] = df['release_date'].dt.year
df['release_month'] = df['release_date'].dt.month

# Mengubah Bahasa (teks) menjadi Angka
le = LabelEncoder()
df['original_language_encoded'] = le.fit_transform(df['original_language'])

# Menentukan Fitur (X) dan Target (y)
features = ['popularity', 'vote_count', 'release_year', 'release_month', 'original_language_encoded']
X = df[features]
y = df['vote_average']

# Membagi Data (80% Latih, 20% Uji)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ==========================================
# 2. TRAINING MODEL (RANDOM FOREST)
# ==========================================
print("Sedang melatih model Random Forest (100 pohon)...")
# Kita menggunakan algoritma JUARA kita
model = RandomForestRegressor(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# ==========================================
# 3. EVALUASI LENGKAP
# ==========================================
print("\n" + "="*40)
print("HASIL EVALUASI MODEL")
print("="*40)

# Prediksi data test
y_pred = model.predict(X_test)

# Menghitung Metrik
r2 = r2_score(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
mae = mean_absolute_error(y_test, y_pred)

print(f"1. Single Split Test:")
print(f"   - R2 Score (Akurasi) : {r2:.4f} ({r2*100:.2f}%)")
print(f"   - RMSE (Error Kuadrat): {rmse:.4f}")
print(f"   - MAE (Rata-rata Error): {mae:.4f} poin")

# Cross Validation (5-Fold)
print(f"\n2. Cross Validation (5-Fold):")
cv_scores = cross_val_score(model, X, y, cv=5, scoring='r2')
print(f"   - Skor per tes : {cv_scores}")
print(f"   - Rata-rata R2 : {cv_scores.mean():.4f}")
print(f"   - Kestabilan   : +/- {cv_scores.std():.4f}")

# ==========================================
# 4. FEATURE IMPORTANCE (RAHASIA MODEL)
# ==========================================
print("\n" + "="*40)
print("FAKTOR PENENTU RATING (Feature Importance)")
print("="*40)

importances = model.feature_importances_
feature_importance_df = pd.DataFrame({'Fitur': features, 'Kepentingan': importances})
feature_importance_df = feature_importance_df.sort_values(by='Kepentingan', ascending=False)

print(feature_importance_df.to_string(index=False))

# Contoh Prediksi
print("\n" + "="*40)
print(f"Contoh Prediksi: Rating Asli {y_test.iloc[0]} | Prediksi Model {y_pred[0]:.2f}")

Sedang memproses data...
Sedang melatih model Random Forest (100 pohon)...

HASIL EVALUASI MODEL
1. Single Split Test:
   - R2 Score (Akurasi) : 0.8569 (85.69%)
   - RMSE (Error Kuadrat): 0.9084
   - MAE (Rata-rata Error): 0.5492 poin

2. Cross Validation (5-Fold):
   - Skor per tes : [0.54879999 0.77793875 0.82384791 0.84153289 0.8144776 ]
   - Rata-rata R2 : 0.7613
   - Kestabilan   : +/- 0.1083

FAKTOR PENENTU RATING (Feature Importance)
                    Fitur  Kepentingan
               vote_count     0.833263
               popularity     0.066340
             release_year     0.048906
            release_month     0.027821
original_language_encoded     0.023670

Contoh Prediksi: Rating Asli 7.0 | Prediksi Model 5.17
